## Notebook to Analyze whether repeating place fields spatially cluster on the maze
### WH 2023-10-13

In [ ]:
%connect_info

In [119]:
# Imports 
import numpy as np
import utility_fx as util
import os
from matplotlib import pyplot as plt
import ratterdam_Defaults as Def
import ratterdam_visBasic as Vis
import ratterdam_RepetitionCoreFx as RepCore
import RateMapClass_William_20190308 as RateMapClass
import williamDefaults as wmDef
import alleyTransitions as alleyTrans
import newAlleyBounds as nab
import pandas as pd
import repeatingPC as repPC
import ratterdam_DataFiltering as Filt 
import pickle 
from matplotlib import path
import itertools
from scipy.stats import pearsonr

In [7]:
## Reading in file with all data
with open("E:\\Ratterdam\\R_data_repetition\\20220405-124315_superPopulationRepetition.pickle","rb") as f:
    alldat = pickle.load(f)  

In [25]:
rat, day = "R859", "D1"
data = alldat[rat][day]
units = data['units']

In [26]:
clustList = list(units.keys())

In [30]:
ratborders = nab.loadAlleyBounds(rat, day)


In [106]:
alleyDistances  = pd.read_excel("E:\\Ratterdam\\alley_distances_full.xlsx",index_col=0)

In [93]:
def calcSingleAlleyRatemap(unit,alley):
    ontrackSpikesIdx = np.empty((0), dtype=int)
    ontrackPositionIdx = np.empty((0), dtype=int)
    sigma = 1
    b  = ratborders.alleyInterBounds[str(alley)]
    ul, ll, ur, lr = [b[0][0], b[1][1]], \
                    [b[0][0], b[1][0]], \
                    [b[0][1], b[1][1]], \
                    [b[0][1], b[1][0]]

    contour = path.Path([ll, lr, ur, ul])
    posIdx = np.where(contour.contains_points(unit.position[:,1:]))[0]
    spkIdx = np.where(contour.contains_points(unit.spikes[:,1:]))[0]

    ontrackPositionIdx = np.concatenate((ontrackPositionIdx, 
                                        posIdx.astype(int)))

    ontrackSpikesIdx = np.concatenate((ontrackSpikesIdx, 
                                        spkIdx.astype(int)))
    position = unit.position[ontrackPositionIdx]
    spikes = unit.spikes[ontrackSpikesIdx]


    if alley in [0, 4, 6, 1, 12, 8, 15, 13, 10]:
        rows,cols = [5,8]
        dim = 0 # axis to average over - long axis. 
    elif alley in [2, 16, 3, 14, 5, 11, 7, 9]:
        rows, cols = [8,5]
        dim = 1

    hs,xe,ye = np.histogram2d(spikes[:,2],spikes[:,1],bins=[rows, cols])
    ho = np.histogram2d(position[:,2],position[:,1],bins=[rows, cols])[0]
    n = (hs*np.reciprocal(ho))*30
    n[np.where(ho==0)] = np.nan


    sm_n = util.weird_smooth(np.nanmean(n,axis=dim),sigma)

    return sm_n


In [112]:
pairs = list(itertools.combinations(alleyDistances.columns,2))

In [123]:
maxD = alleyDistances.max().max() # this should always be 4. 
corrByDist = {i:[] for i in range(1,maxD+1)}

In [124]:
for (a1,a2) in pairs:
    n1 = calcSingleAlleyRatemap(unit,a1)
    n2 = calcSingleAlleyRatemap(unit,a2)
    r,p = pearsonr(n1,n2)
    d = alleyDistances[a1][a2]
    corrByDist[d].append(r)
    

<ipython-input-93-54855bcd8b71>:33: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
<ipython-input-93-54855bcd8b71>:33: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30


In [129]:
meancorr = [np.nanmean(i) for i in corrByDist.values()]
meancorrs.append(meancorr)

## Whole routine below

In [135]:
meancorrs = []

rat_list = ['R765',
            'R765',
            'R781', 
            'R781', 
            'R808', 
            'R808', 
            'R859', 
            'R859', 
            'R886', 
            'R886']

day_list = ['RFD5',
            'DFD4',
            'D3', 
            'D4',
            'D6',
            'D7',
            'D1',
            'D2',
            'D1',
            'D2']





In [155]:
all_meancorrs = []
for rat, day in zip(rat_list, day_list):
     print(rat,day)
     ratborders = nab.loadAlleyBounds(rat, day)
     data = alldat[rat][day]
     units = data['units']
     
     clustList = list(units.keys())

     for unitName, unit in units.items():
          if unit.repeating:
               corrByDist = {i:[] for i in range(1,maxD+1)}
               for (a1,a2) in pairs:
                    n1 = calcSingleAlleyRatemap(unit,a1)
                    n2 = calcSingleAlleyRatemap(unit,a2)
                    r,p = pearsonr(n1,n2)
                    d = alleyDistances[a1][a2]
                    corrByDist[d].append(r)
               meancorr = [np.nanmean(i) for i in corrByDist.values()]
               all_meancorrs.append(meancorr)
    
     

R765 RFD5


<ipython-input-93-54855bcd8b71>:33: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
<ipython-input-93-54855bcd8b71>:33: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
c:\Users\whockei1\Anaconda3\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


R765 DFD4
R781 D3
R781 D4
R808 D6
R808 D7
R859 D1
R859 D2
R886 D1
R886 D2
